In [1]:
import json
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
from dotenv import load_dotenv

In [2]:
# es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
es = Elasticsearch("http://localhost:9200")

## Q1

In [ ]:
# !curl localhost:9200

{
  "name" : "ebc281b872c0",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "smOS2ZgAQJW04BUPdNyL8A",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   539  100   539    0     0   2189      0 --:--:-- --:--:-- --:--:--  2200


In [4]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [8]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [10]:
index_setting = {
    "settings": {
        "number_of_shards":1,
        "number_of_replicas":0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question":{"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

index_name = "course-assignment"
es.indices.create(index=index_name, body=index_setting)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-assignment'})

## Q2

In [11]:
# es.info()
for doc in tqdm(documents):
    es.index(index=index_name, document=doc)
  

  0%|          | 0/948 [00:00<?, ?it/s]

## Q3 - 4

In [132]:
# query = "How do execute a command on a Kubernetes pod?"
query = "How do copy a file to a Docker container?"

In [208]:
def elastic_search(query):
    search_query ={
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
            }
        }
    }
    
    response = es.search(index=index_name, body=search_query)
    
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return {"result": result_docs, "raw_response": response}


In [ ]:
res = elastic_search(query=query)

In [210]:
res['result']

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

## Q5

In [211]:
context_template = """
Q: {question}
A: {text}
""".strip()

context_template

'Q: {question}\nA: {text}'

In [212]:
context_template = """
Q: {question}
A: {text}
""".strip()

context_entries = []
for record in res['result']:
    context_entries.append(context_template.format(question=record["question"], text=record["text"]))

# Join all context entries with two line breaks
context = "\n\n".join(context_entries)

# Print the final context
print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [ ]:
#  Important note:
""""
The question asks us to use this query: {"How do I execute a command in a running docker container?"}, 
but the exercise does not give the correct answer except we use the previous query: {"How do copy a file to a Docker container?"}.
This should be corrected in the document.
"""
q = "How do I execute a command in a running docker container?"

In [213]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [218]:
prompt = prompt_template.format(question=query, context=context)

print(f"prompt length is: {len(prompt)}")

prompt length is: 1446


## Q6

In [221]:
# !pip install tiktoken
import tiktoken

In [222]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [225]:
encoded_prompt = encoding.encode(prompt)
len(encoded_prompt)

320